Author: Nelson Liu

Email: [nliu@uncharted.software](mailto:nliu@uncharted.software)

# SIR-Age Stratification & MIRA Reconstruction Experiment

Simple stratification: SIR model x 2-age contact model -> MIRA semantic reconstruction

To be compared with MIRA stratification.

In [14]:
# Libraries
using AlgebraicPetri, AlgebraicPetri.TypedPetri
using Catlab.Programs, Catlab.Graphics
using Catlab.CategoricalAlgebra
using JSON
# using GraphViz

## Define Ontology

In [6]:
const infectious_ontology = LabelledPetriNet(
    [:Pop],
    :infect => ((:Pop, :Pop) => (:Pop, :Pop)),
    :disease => (:Pop => :Pop),
    :strata => (:Pop => :Pop)
)

# Graph(infectious_ontology)

LabelledPetriNet with elements T = 1:3, S = 1:1, I = 1:4, O = 1:4, Name = 1:0
┌───┬─────────┐
│ T │   tname │
├───┼─────────┤
│ 1 │  infect │
│ 2 │ disease │
│ 3 │  strata │
└───┴─────────┘
┌───┬───────┐
│ S │ sname │
├───┼───────┤
│ 1 │   Pop │
└───┴───────┘
┌───┬────┬────┐
│ I │ it │ is │
├───┼────┼────┤
│ 1 │  1 │  1 │
│ 2 │  1 │  1 │
│ 3 │  2 │  1 │
│ 4 │  3 │  1 │
└───┴────┴────┘
┌───┬────┬────┐
│ O │ ot │ os │
├───┼────┼────┤
│ 1 │  1 │  1 │
│ 2 │  1 │  1 │
│ 3 │  2 │  1 │
│ 4 │  3 │  1 │
└───┴────┴────┘

## Define Base Models

In [8]:
# SIR model
sir_uwd = @relation () where (S::Pop, I::Pop, R::Pop) begin
    infect(S, I, I, I)
    disease(I, R)
end

to_graphviz(sir_uwd, box_labels = :name, junction_labels = :variable)

tnames = [:beta, :gamma]
typed_sir = oapply_typed(infectious_ontology, sir_uwd, tnames)

# Graph(typed_sir)

ACSetTransformation((T = FinFunction([1, 2], 2, 3), S = FinFunction([1, 1, 1], 3, 1), I = FinFunction([1, 2, 3], 3, 4), O = FinFunction([1, 2, 3], 3, 4), Name = LooseVarFunction{Symbol, Symbol}(FinDomFunction(Union{AttrVar, Symbol}[], FinSet(0), TypeSet(Union{AttrVar, Symbol})), SetFunction(#10, TypeSet(Symbol), TypeSet(Symbol)), FinSet(0))), LabelledPetriNet {T = 2, S = 3, I = 3, O = 3, Name = 0}, LabelledPetriNet {T = 3, S = 1, I = 4, O = 4, Name = 0})

In [9]:
# Age (contact) model
N = 2
snames = [Symbol("A$i") for i in 1:N]

typed_age = pairwise_id_typed_petri(infectious_ontology, :Pop, :infect, snames)

# Graph(typed_age)

ACSetTransformation((T = FinFunction([1, 1, 1, 1], 4, 3), S = FinFunction([1, 1], 2, 1), I = FinFunction([1, 2, 1, 2, 1, 2, 1, 2], 8, 4), O = FinFunction([1, 2, 1, 2, 1, 2, 1, 2], 8, 4), Name = LooseVarFunction{Symbol, Symbol}(FinDomFunction(Union{AttrVar, Symbol}[], FinSet(0), TypeSet(Union{AttrVar, Symbol})), SetFunction(#33, TypeSet(Symbol), TypeSet(Symbol)), FinSet(0))), LabelledPetriNet {T = 4, S = 2, I = 8, O = 8, Name = 0}, LabelledPetriNet {T = 3, S = 1, I = 4, O = 4, Name = 0})

## Stratification

In [11]:
# Augment SIR model
typed_sir_aug = add_reflexives(
    typed_sir,
    [[], [], []],
    # [[:strata], [:strata], [:strata]],
    infectious_ontology
)

# Graph(dom(typed_sir_aug))

ACSetTransformation((T = FinFunction([1, 2], 2, 3), S = FinFunction([1, 1, 1], 3, 1), I = FinFunction([1, 2, 3], 3, 4), O = FinFunction([1, 2, 3], 3, 4), Name = LooseVarFunction{Symbol, Symbol}(FinDomFunction(Union{AttrVar, Symbol}[], FinSet(0), TypeSet(Union{AttrVar, Symbol})), SetFunction(#17, TypeSet(Symbol), TypeSet(Symbol)), FinSet(0))), LabelledPetriNet {T = 2, S = 3, I = 3, O = 3, Name = 0}, LabelledPetriNet {T = 3, S = 1, I = 4, O = 4, Name = 0})

In [12]:
# Augment age (contact) model
# Permit all age states to have a "disease" self-interaction
# (edit: need to add "strata" self-interaction now, not after 1st stratification)
typed_age_aug = add_reflexives(
    typed_age,
    repeat([[:disease]], N),
    # repeat([[:disease, :strata]], N),
    infectious_ontology
)

# Graph(dom(typed_age_aug))
# Graph(typed_age_aug)

ACSetTransformation((T = FinFunction([1, 1, 1, 1, 2, 2], 6, 3), S = FinFunction([1, 1], 2, 1), I = FinFunction([1, 2, 1, 2, 1, 2, 1, 2, 3, 3], 10, 4), O = FinFunction([1, 2, 1, 2, 1, 2, 1, 2, 3, 3], 10, 4), Name = LooseVarFunction{Symbol, Symbol}(FinDomFunction(Union{AttrVar, Symbol}[], FinSet(0), TypeSet(Union{AttrVar, Symbol})), SetFunction(#17, TypeSet(Symbol), TypeSet(Symbol)), FinSet(0))), LabelledPetriNet {T = 6, S = 2, I = 10, O = 10, Name = 0}, LabelledPetriNet {T = 3, S = 1, I = 4, O = 4, Name = 0})

In [13]:
# Stratify SIR and Age models
typed_sir_age = typed_product(typed_sir_aug, typed_age_aug)

# Graph(dom(typed_sir_age))

ACSetTransformation((T = compose(compose(FinFunction([1, 3, 5, 7, 10, 12], 6, 12), FinFunction(#43, FinSet(12), FinSet(2))), FinFunction([1, 2], 2, 3)), S = compose(compose(FinFunction([1, 2, 3, 4, 5, 6], 6, 6), FinFunction(#43, FinSet(6), FinSet(3))), FinFunction([1, 1, 1], 3, 1)), I = compose(compose(FinFunction([1, 5, 7, 11, 13, 17, 19, 23, 27, 30], 10, 30), FinFunction(#43, FinSet(30), FinSet(3))), FinFunction([1, 2, 3], 3, 4)), O = compose(compose(FinFunction([1, 5, 7, 11, 13, 17, 19, 23, 27, 30], 10, 30), FinFunction(#43, FinSet(30), FinSet(3))), FinFunction([1, 2, 3], 3, 4)), Name = LooseVarFunction{Tuple{Symbol, Symbol}, Symbol}(FinDomFunction(Union{AttrVar, Symbol}[], FinSet(0), TypeSet(Union{AttrVar, Symbol})), compose(SetFunction(#4, TypeSet(Tuple{Symbol, Symbol}), TypeSet(Symbol)), SetFunction(#17, TypeSet(Symbol), TypeSet(Symbol))), FinSet(0))), AlgebraicPetri.LabelledPetriNetUntyped{Tuple{Symbol, Symbol}} {T = 6, S = 6, I = 10, O = 10, Name = 0}, LabelledPetriNet {T = 3, 

In [19]:
# Flatten the labels
typed_sir_age_flat = flatten_labels(typed_sir_age)

ACSetTransformation((T = compose(compose(FinFunction([1, 3, 5, 7, 10, 12], 6, 12), FinFunction(#43, FinSet(12), FinSet(2))), FinFunction([1, 2], 2, 3)), S = compose(compose(FinFunction([1, 2, 3, 4, 5, 6], 6, 6), FinFunction(#43, FinSet(6), FinSet(3))), FinFunction([1, 1, 1], 3, 1)), I = compose(compose(FinFunction([1, 5, 7, 11, 13, 17, 19, 23, 27, 30], 10, 30), FinFunction(#43, FinSet(30), FinSet(3))), FinFunction([1, 2, 3], 3, 4)), O = compose(compose(FinFunction([1, 5, 7, 11, 13, 17, 19, 23, 27, 30], 10, 30), FinFunction(#43, FinSet(30), FinSet(3))), FinFunction([1, 2, 3], 3, 4)), Name = LooseVarFunction{Tuple{Symbol, Symbol}, Symbol}(FinDomFunction(Union{AttrVar, Symbol}[], FinSet(0), TypeSet(Union{AttrVar, Symbol})), compose(SetFunction(#4, TypeSet(Tuple{Symbol, Symbol}), TypeSet(Symbol)), SetFunction(#17, TypeSet(Symbol), TypeSet(Symbol))), FinSet(0))), LabelledPetriNet {T = 6, S = 6, I = 10, O = 10, Name = 0}, LabelledPetriNet {T = 3, S = 1, I = 4, O = 4, Name = 0})

In [21]:
write_json_acset(typed_sir_age_flat, "sir_age_model.json")

MethodError: MethodError: no method matching write_json_acset(::Catlab.CategoricalAlgebra.CSets.StructLooseACSetTransformation{TypeLevelBasicSchema{Symbol, Tuple{:T, :S, :I, :O}, Tuple{(:it, :I, :T), (:is, :I, :S), (:ot, :O, :T), (:os, :O, :S)}, Tuple{:Name}, Tuple{(:tname, :T, :Name), (:sname, :S, :Name)}}, NamedTuple{(:T, :S, :I, :O, :Name), Tuple{Catlab.CategoricalAlgebra.Sets.CompositeFunction{Catlab.CategoricalAlgebra.FinSets.FinSetInt, Catlab.CategoricalAlgebra.FinSets.FinSetInt, Catlab.CategoricalAlgebra.Sets.CompositeFunction{Catlab.CategoricalAlgebra.FinSets.FinSetInt, Catlab.CategoricalAlgebra.FinSets.FinSetInt, Catlab.CategoricalAlgebra.FinSets.FinDomFunctionVector{Int64, Vector{Int64}, Catlab.CategoricalAlgebra.FinSets.FinSetInt}, Catlab.CategoricalAlgebra.Sets.SetFunctionCallable{Int64, Int64, Catlab.CategoricalAlgebra.FinSets.FinSetInt, Catlab.CategoricalAlgebra.FinSets.FinSetInt}}, Catlab.CategoricalAlgebra.FinSets.FinDomFunctionVector{Int64, Vector{Int64}, Catlab.Catego

In [22]:
test = StratifiedASKEMPetriNet(typed_sir_aug, typed_age_aug)

UndefVarError: UndefVarError: `StratifiedASKEMPetriNet` not defined